# Notebook 5: Inferencing with Deployed Model

### In this notebook, we deploy the model previously logged to MLflow and make some predictions to demonstrate that it works. We also transform some of the data in the test set to practice model monitoring in Databricks.

In [0]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

In [0]:
# Define path 
input_path = "../data/ames_preprocessed_numeric_unscaled.csv"

# Check if file exists to avoid path errors
if not os.path.exists(input_path):
    print(f"⚠️ Warning: File not found at {input_path}")
    print("Trying absolute path or checking current directory...")
    # Fallback: check if the file is in the same folder
    input_path = "ames_preprocessed_numeric_unscaled.csv"

# Load the CSV
df = pd.read_csv(input_path)

# Define target column
target = "SalePrice_log"

# Split features and target
X = df.drop(columns=[target])
y = df[target]

# Split into train/test sets (MUST use same random_state=42 as training!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ Data loaded! X_test shape: {X_test.shape}")

In [0]:
# This is a model monitoring function, in lieu of proper model monitoring functionality in the databricks free edition

def monitor_model_performance(model_alias, X_data, y_true_log, data_name="Test Data", rmse_threshold=40000):
    """
    Simulates a Model Monitoring Job.
    Converts LOG predictions back to DOLLARS for a realistic report.
    """
    print(f"\n{'='*60}")
    print(f"📡 STARTING MONITORING JOB: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🎯 Target Model Alias: @{model_alias}")
    print(f"📊 Data Source: {data_name}")
    print(f"{'='*60}")

    # 1. Load the "Production" Model
    model_uri = f"models:/{model_name}@{model_alias}"
    production_model = mlflow.sklearn.load_model(model_uri)
    
    # 2. Run Inference
    print(f"   🤖 Running inference on {len(X_data)} samples...")
    log_predictions = production_model.predict(X_data)
    
    # 3. CONVERT TO DOLLARS (Fixing the unit mismatch)
    # np.expm1 reverses the log(x+1) transformation
    pred_dollars = np.expm1(log_predictions)
    actual_dollars = np.expm1(y_true_log)
    
    # 4. Calculate RMSE in Real Dollars
    rmse_dollars = np.sqrt(((pred_dollars - actual_dollars) ** 2).mean())
    
    # 5. Generate Monitoring Report
    print(f"\n📋 --- MONITORING REPORT ---")
    print(f"   ✅ Metric Evaluated: RMSE (Dollars)")
    print(f"   📉 Current Performance: ${rmse_dollars:,.2f}")
    print(f"   📏 Threshold Limit:     ${rmse_threshold:,.2f}")
    
    # 6. Status Check
    if rmse_dollars < rmse_threshold:
        print(f"\n   🟢 STATUS: HEALTHY. Model is performing within expected limits.")
    else:
        print(f"\n   🔴 STATUS: ALERT! Model performance has degraded.")
        print(f"      Action: Trigger retraining or investigate data drift.")
        
    print(f"{'='*60}\n")
    return rmse_dollars

In [0]:


# 1. Construct the Model URI
# IMPORTANT: Use the EXACT name you see in the "Models" UI (e.g., "main.default.rf_ames_model")
model_name = "rf_ames_model" 
model_uri = f"models:/{model_name}@prod"

print(f"Attempting to load model from: {model_uri}")

# 2. Load the model
# We use sklearn.load_model because that's how we saved it
production_model = mlflow.sklearn.load_model(model_uri)

print("✅ Model loaded successfully!")

# 3. (Optional) Inspect the model to be sure
print(f"Model type: {type(production_model)}")

# Validate Deployed Model on Normal Test Data 

In [0]:
# --- INFERENCE DEMO ---
print("🚀 Loading Deployed Model (@prod)...")
model_uri = f"models:/{model_name}@prod"
production_model = mlflow.sklearn.load_model(model_uri)

# 1. Select 5 random houses from the test set
sample_data = X_test.sample(5, random_state=42)
sample_indices = sample_data.index

# 2. Get the Actual Prices (and convert from Log back to Dollars)
actual_log = y_test.loc[sample_indices]
actual_prices = np.expm1(actual_log)

# 3. Generate Predictions (and convert from Log back to Dollars)
print(f"🔮 Predicting prices for {len(sample_data)} houses...")
pred_log = production_model.predict(sample_data)
pred_prices = np.expm1(pred_log)

# 4. Create a Comparison Table
results_df = pd.DataFrame({
    'Actual Price': actual_prices,
    'Predicted Price': pred_prices,
    'Difference ($)': pred_prices - actual_prices,
    'Error %': np.abs((pred_prices - actual_prices) / actual_prices) * 100
})

# Formatting for nice display
pd.options.display.float_format = '${:,.2f}'.format
print("\n✨ INFERENCE RESULTS (Sample):")
display(results_df)

# Reset formatting (optional)
pd.reset_option('display.float_format')

In [0]:
# Set up model monitoring by calling function
monitor_model_performance(
    model_alias="prod", 
    X_data=X_test, 
    y_true_log=y_test, 
    data_name="Standard Holdout Test Set",
    rmse_threshold=40000
)

# Feature Changes

## This is done to test out the modeling monitoring dashboard in Databricks


In [0]:
# --- STEP 8: Simulate Data Drift (The "Broken" Pipeline) ---
print("⚠️ INJECTING DATA DRIFT...")
X_test_drifted = X_test.copy()

# SCENARIO 1: Data Entry Error (Garage Area becomes 5x larger)
# Using 'Garage Area' (with space) based on your feature list
X_test_drifted['Garage Area'] = X_test_drifted['Garage Area'] * 5 

# SCENARIO 2: Sensor Failure (Living Area reads as 0)
# Using 'Gr Liv Area' (with space) based on your feature list
X_test_drifted['Gr Liv Area'] = 0

print("Data corruption complete. Sending to production model...")

In [0]:
# --- STEP 8: Simulate CATASTROPHIC Data Drift ---
print("⚠️ INJECTING CATASTROPHIC DATA DRIFT...")
X_test_drifted = X_test.copy()

# GO NUCLEAR: Multiply ALL data by 100
# This simulates a currency/unit conversion error affecting the whole dataset
X_test_drifted = X_test_drifted * 100

print("Data is now completely broken. Sending to production model...")

## Verify data drift through model monitoring function

In [0]:
# --- STEP 9: Verify Monitoring Catch (The "Red" Status) ---
monitor_model_performance(
    model_alias="prod", 
    X_data=X_test_drifted, 
    y_true_log=y_test, 
    data_name="🔥 COMPLETELY BROKEN DATA",
    rmse_threshold=40000 
)